In [3]:
import tsfresh
import pandas
import os
import numpy as np
from imageio import imread
from tqdm import tqdm
from tsfresh.feature_extraction import ComprehensiveFCParameters


In [2]:
data = pd.read_csv("data.csv") 
data.head()

In [3]:
col_features = ['f'+str(i) for i in range(nb_colonnes_dataframe)]
col_names =  ['id', 'time'] + col_features

In [4]:
images = []
y = {}

paths = ['frames/train/malfunction/', 'frames/train/stable/']
counter = 0
for i in range(len(paths)):
    files = [f for f in os.listdir(paths[i]) if f.lower().endswith(".png")]
    for file in files:
        images.append(os.path.join(paths[i], file))
        y[counter] = i
        counter += 1

In [5]:
my_df  = pandas.DataFrame(columns = col_names)

for i in tqdm(range(len(images))): 
    image = images[i]
    rgb_matrix = imread(image)
    df_image = pandas.DataFrame(rgb_matrix[:, :, 0], columns = col_features)
    df_image['time'] = range(nb_time)
    df_image['id'] = i
    my_df = my_df.append(df_image)
    
my_df = my_df[my_df.columns[::-1]]

  0%|          | 0/12 [00:00<?, ?it/s]C:\Users\feor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:7134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(
100%|██████████| 12/12 [00:00<00:00, 24.00it/s]


In [6]:
my_df

,time,id,f99,f98,f97,f96,f95,f94,f93,f92,...,f106,f105,f104,f103,f102,f101,f100,f10,f1,f0
0,0,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,1,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,2,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,3,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,4,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,475,11,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
476,476,11,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
477,477,11,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
478,478,11,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [7]:
my_df.astype('float32').dtypes

time    float32
id      float32
f99     float32
f98     float32
f97     float32
         ...   
f101    float32
f100    float32
f10     float32
f1      float32
f0      float32
Length: 642, dtype: object

In [12]:
extraction_settings = ComprehensiveFCParameters()

features_filtered_direct = tsfresh.extract_relevant_features(my_df.astype('float32'), pandas.Series(y),
                                                     column_id='id', column_sort='time', default_fc_parameters=extraction_settings)

Feature Extraction: 100%|██████████| 15/15 [15:03<00:00, 60.24s/it]


In [14]:
features_filtered_direct

variable
id
0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
